In [1]:
from langchain.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore.document import Document

In [2]:
def open_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            contents = file.read()
        return contents
    
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
GOOGLE_API_KEY = ''

In [ ]:
paths = [
    "financial_agents\knowledge\dados_fundamentalistas.txt", 
    "financial_agents\knowledge\glossario_financias.txt"
]

pages = []

for path in paths:
    loader = open_file(path)
    if loader:
        pages.append(loader)

In [5]:
size = 1000
overlap = 200
persist_directory = "db"

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=GOOGLE_API_KEY)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=size,
    chunk_overlap=overlap
)

In [7]:
documents = []
for page in pages:
    chunks = text_splitter.split_text(page)
    for chunk in chunks:
        documents.append(Document(page_content=chunk))    

In [8]:
vector_store = FAISS.from_documents(documents=documents, embedding=embeddings)

In [9]:
vector_store.save_local("db")

In [ ]:
question = "O que é uma Ebitda?"

retriever = vector_store.as_retriever()

docs = retriever.get_relevant_documents(question)

In [10]:
docs_scores = vector_store.similarity_search_with_score(question, k=2)
for doc, score in docs_scores:
    print(f"\n📄 Score: {score:.4f}")
    print(doc.page_content[:300])


📄 Score: 0.2736
Ebitda: Este indicador representa, em tradução livre do inglês, o lucro antes de juros, impostos, depreciação e amortização. É a partir do Ebitda que analistas e investidores medem o potencial de geração de caixa de uma companhia, uma vez que ele mostra os valores obtidos pelo negócio excluindo os c

📄 Score: 0.3252
EBITDA: Sigla em inglês para Earnings Before Interests, Taxes, Depreciation and Amortization, ou em português Lucro antes dos Juros, Impostos, Depreciação e Amortização (LAJIDA), é um indicativo do potencial de geração de caixa operacional da empresa, sem levar em consideração os efeitos de receitas


In [11]:
docs = vector_store.similarity_search(question)

context = ""
for doc in docs:
    context += doc.page_content 
    context += "\n\n"

In [12]:
context

'Ebitda: Este indicador representa, em tradução livre do inglês, o lucro antes de juros, impostos, depreciação e amortização. É a partir do Ebitda que analistas e investidores medem o potencial de geração de caixa de uma companhia, uma vez que ele mostra os valores obtidos pelo negócio excluindo os custos que não estão ligados à operação.\n\nMargem Ebitda: A margem Ebitda é um indicador de margem de lucro operacional de um negócio. Se obtém este indicador pela divisão entre o Ebitda registrado pela companhia em determinado período e as receitas. Quanto maior a margem Ebitda, maior é a eficiência operacional da empresa, uma vez que ela consegue gerar lucro a partir de suas operações.\n\nPatrimônio Líquido (PL): Este indicador representa a diferença entre o valor dos ativos de uma empresa e o seu passivo. Lembrando que ativos são os bens e direitos a receber pela companhia, e os passivos as obrigações com terceiros e com os sócios.\n\nEBITDA: Sigla em inglês para Earnings Before Interest